In [32]:
import pandas as pd
from nba_api.stats.endpoints import *
import json
import time

In [34]:
# Define the playtypes and seasons
playtypes = ['Isolation', 'Transition', 'PRBallHandler', 'PRRollman', 'Postup', 'Spotup', 'Handoff', 'Cut', 'OffScreen', 'OffRebound', 'Misc']
seasons = ['2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24', '2024-25']

# Columns that shouldn't be renamed with playtype suffix
id_columns = ['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'TYPE_GROUPING']

# Initialize an empty list to store all dataframes
all_dfs = []

# Loop through each season and playtype
for season in seasons:
    print(f"Processing season: {season}")
    
    season_dfs = []
    
    for playtype in playtypes:
        print(f"  - Getting data for playtype: {playtype}")
        
        try:
            time.sleep(1)
            # API call
            synergy_data = synergyplaytypes.SynergyPlayTypes(
                league_id="00",
                per_mode_simple="PerGame",
                play_type_nullable=playtype,
                player_or_team_abbreviation="P",
                season=season,
                season_type_all_star="Regular Season",
                type_grouping_nullable="Offensive", timeout=2
            )
            
            # Convert to dataframe
            data = json.loads(synergy_data.get_json())
            df = pd.DataFrame(data['resultSets'][0]['rowSet'], columns=data['resultSets'][0]['headers'])
            
            # Remove 'PLAY_TYPE' from the input data since we'll create our own playtype-specific columns
            if 'PLAY_TYPE' in df.columns:
                df = df.drop(columns=['PLAY_TYPE'])
            
            # Rename stat columns to include playtype
            rename_dict = {}
            for col in df.columns:
                if col not in id_columns:
                    rename_dict[col] = f"{col}_{playtype}"
            
            df = df.rename(columns=rename_dict)
            
            # Add to list of dataframes for this season
            season_dfs.append(df)
            
            print(f"    Successfully retrieved {len(df)} player records")
            
        except Exception as e:
            print(f"    Error retrieving {playtype} data for {season}: {str(e)}")
    
    # If we have dataframes for this season, merge them
    if season_dfs:
        # Start with the first dataframe
        season_df = season_dfs[0]
        
        # Merge with the rest
        for df in season_dfs[1:]:
            season_df = pd.merge(
                season_df,
                df,
                on=['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'TYPE_GROUPING'],
                how='outer'
            )
        
        # Add to list of all seasons
        all_dfs.append(season_df)
        print(f"  Completed merging {len(season_dfs)} playtypes for season {season}")
    
# Combine all seasons
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    print(f"Final dataframe shape: {final_df.shape}")
    
    # Fill NaN values for missing data
    print("Filling NaN values with appropriate defaults...")
    
    # Save to CSV
    final_df.to_csv('synergy_all_playtypes_2015_to_2025.csv', index=False)
    print("Data saved to 'synergy_all_playtypes_2015_to_2025.csv'")
else:
    print("No data was retrieved.")

# Display the first few rows
final_df.head()

Processing season: 2015-16
  - Getting data for playtype: Isolation
    Successfully retrieved 287 player records
  - Getting data for playtype: Transition
    Successfully retrieved 396 player records
  - Getting data for playtype: PRBallHandler
    Successfully retrieved 272 player records
  - Getting data for playtype: PRRollman
    Successfully retrieved 202 player records
  - Getting data for playtype: Postup
    Successfully retrieved 216 player records
  - Getting data for playtype: Spotup
    Successfully retrieved 392 player records
  - Getting data for playtype: Handoff
    Successfully retrieved 233 player records
  - Getting data for playtype: Cut
    Successfully retrieved 334 player records
  - Getting data for playtype: OffScreen
    Successfully retrieved 238 player records
  - Getting data for playtype: OffRebound
    Successfully retrieved 287 player records
  - Getting data for playtype: Misc
    Successfully retrieved 366 player records
  Completed merging 11 playty

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,TYPE_GROUPING,PERCENTILE_Isolation,GP_Isolation,POSS_PCT_Isolation,PPP_Isolation,FG_PCT_Isolation,FT_POSS_PCT_Isolation,TOV_POSS_PCT_Isolation,SF_POSS_PCT_Isolation,PLUSONE_POSS_PCT_Isolation,SCORE_POSS_PCT_Isolation,EFG_PCT_Isolation,POSS_Isolation,PTS_Isolation,FGM_Isolation,FGA_Isolation,FGMX_Isolation,PERCENTILE_Transition,GP_Transition,POSS_PCT_Transition,PPP_Transition,FG_PCT_Transition,FT_POSS_PCT_Transition,TOV_POSS_PCT_Transition,SF_POSS_PCT_Transition,PLUSONE_POSS_PCT_Transition,SCORE_POSS_PCT_Transition,EFG_PCT_Transition,POSS_Transition,PTS_Transition,FGM_Transition,FGA_Transition,FGMX_Transition,PERCENTILE_PRBallHandler,GP_PRBallHandler,POSS_PCT_PRBallHandler,PPP_PRBallHandler,FG_PCT_PRBallHandler,FT_POSS_PCT_PRBallHandler,TOV_POSS_PCT_PRBallHandler,SF_POSS_PCT_PRBallHandler,PLUSONE_POSS_PCT_PRBallHandler,SCORE_POSS_PCT_PRBallHandler,EFG_PCT_PRBallHandler,POSS_PRBallHandler,PTS_PRBallHandler,FGM_PRBallHandler,FGA_PRBallHandler,FGMX_PRBallHandler,PERCENTILE_PRRollman,GP_PRRollman,POSS_PCT_PRRollman,PPP_PRRollman,FG_PCT_PRRollman,FT_POSS_PCT_PRRollman,TOV_POSS_PCT_PRRollman,SF_POSS_PCT_PRRollman,PLUSONE_POSS_PCT_PRRollman,SCORE_POSS_PCT_PRRollman,EFG_PCT_PRRollman,POSS_PRRollman,PTS_PRRollman,FGM_PRRollman,FGA_PRRollman,FGMX_PRRollman,PERCENTILE_Postup,GP_Postup,POSS_PCT_Postup,PPP_Postup,FG_PCT_Postup,FT_POSS_PCT_Postup,TOV_POSS_PCT_Postup,SF_POSS_PCT_Postup,PLUSONE_POSS_PCT_Postup,SCORE_POSS_PCT_Postup,EFG_PCT_Postup,POSS_Postup,PTS_Postup,FGM_Postup,FGA_Postup,FGMX_Postup,PERCENTILE_Spotup,GP_Spotup,POSS_PCT_Spotup,PPP_Spotup,FG_PCT_Spotup,FT_POSS_PCT_Spotup,TOV_POSS_PCT_Spotup,SF_POSS_PCT_Spotup,PLUSONE_POSS_PCT_Spotup,SCORE_POSS_PCT_Spotup,EFG_PCT_Spotup,POSS_Spotup,PTS_Spotup,FGM_Spotup,FGA_Spotup,FGMX_Spotup,PERCENTILE_Handoff,GP_Handoff,POSS_PCT_Handoff,PPP_Handoff,FG_PCT_Handoff,FT_POSS_PCT_Handoff,TOV_POSS_PCT_Handoff,SF_POSS_PCT_Handoff,PLUSONE_POSS_PCT_Handoff,SCORE_POSS_PCT_Handoff,EFG_PCT_Handoff,POSS_Handoff,PTS_Handoff,FGM_Handoff,FGA_Handoff,FGMX_Handoff,PERCENTILE_Cut,GP_Cut,POSS_PCT_Cut,PPP_Cut,FG_PCT_Cut,FT_POSS_PCT_Cut,TOV_POSS_PCT_Cut,SF_POSS_PCT_Cut,PLUSONE_POSS_PCT_Cut,SCORE_POSS_PCT_Cut,EFG_PCT_Cut,POSS_Cut,PTS_Cut,FGM_Cut,FGA_Cut,FGMX_Cut,PERCENTILE_OffScreen,GP_OffScreen,POSS_PCT_OffScreen,PPP_OffScreen,FG_PCT_OffScreen,FT_POSS_PCT_OffScreen,TOV_POSS_PCT_OffScreen,SF_POSS_PCT_OffScreen,PLUSONE_POSS_PCT_OffScreen,SCORE_POSS_PCT_OffScreen,EFG_PCT_OffScreen,POSS_OffScreen,PTS_OffScreen,FGM_OffScreen,FGA_OffScreen,FGMX_OffScreen,PERCENTILE_OffRebound,GP_OffRebound,POSS_PCT_OffRebound,PPP_OffRebound,FG_PCT_OffRebound,FT_POSS_PCT_OffRebound,TOV_POSS_PCT_OffRebound,SF_POSS_PCT_OffRebound,PLUSONE_POSS_PCT_OffRebound,SCORE_POSS_PCT_OffRebound,EFG_PCT_OffRebound,POSS_OffRebound,PTS_OffRebound,FGM_OffRebound,FGA_OffRebound,FGMX_OffRebound,PERCENTILE_Misc,GP_Misc,POSS_PCT_Misc,PPP_Misc,FG_PCT_Misc,FT_POSS_PCT_Misc,TOV_POSS_PCT_Misc,SF_POSS_PCT_Misc,PLUSONE_POSS_PCT_Misc,SCORE_POSS_PCT_Misc,EFG_PCT_Misc,POSS_Misc,PTS_Misc,FGM_Misc,FGA_Misc,FGMX_Misc
0,22015,708,Kevin Garnett,1610612750,MIN,Minnesota Timberwolves,Offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428,38.0,0.093,1.077,0.600,0.154,0.077,0.154,0.000,0.538,0.600,0.3,0.4,0.2,0.3,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.872,38.0,0.200,1.214,0.630,0.000,0.036,0.000,0.000,0.607,0.630,0.7,0.9,0.4,0.7,0.3,0.749,38.0,0.186,0.923,0.417,0.077,0.000,0.077,0.000,0.462,0.417,0.7,0.6,0.3,0.6,0.4,0.296,38.0,0.257,0.833,0.417,0.000,0.000,0.000,0.000,0.417,0.417,0.9,0.8,0.4,0.9,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026,38.0,0.093,0.846,0.364,0.231,0.000,0.154,0.077,0.385,0.364,0.3,0.3,0.1,0.3,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.131,38.0,0.121,0.235,0.000,0.176,0.824,0.000,0.00,0.176,0.000,0.4,0.1,0.0,0.0,0.0